In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import os
import re
from matplotlib import pyplot as plt
import seaborn as sns
import glob
import re
from scipy.interpolate import interp1d
from tqdm import tqdm, tqdm_notebook
from scipy import spatial
from sklearn.linear_model import LogisticRegression

In [2]:
def calculate_pad(brightness, saturation):
    p = 0.69*brightness + 0.22*saturation
    a = -0.31*brightness + 0.6*saturation
    d = 0.76*brightness + 0.32*saturation
    return [p,a,d]

def calculate_pad_scene(scene):
    pads = []
    for img in scene:
        temp_b = mean_brightness(img)
        temp_s = mean_saturation(img)
        pads.append(calculate_pad(temp_b, temp_s))
    return np.mean([x[0] for x in pads]), np.mean([x[1] for x in pads]), np.mean([x[2] for x in pads])

def calculate_blur(img):
    return cv2.Laplacian(img, cv2.CV_64F).var()

def calculate_blur_scene(scene):
    blurs = []
    for img in scene:
        blurs.append(calculate_blur(img))
    return np.mean(blurs)

def mean_brightness(img):
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV) #convert it to hsv
    return np.mean(hsv[:,:,2])

def mean_saturation(img):
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV) #convert it to hsv
    return np.mean(hsv[:,:,1])

def calculate_opticalFlow(img1, img2):
    prev = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    forward = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
    mask = np.zeros_like(prev)
    mask[..., 1] = 255
    flow = cv2.calcOpticalFlowFarneback(prev, forward, flow=None, pyr_scale=0.5,levels =1,winsize=3,iterations=15,poly_n=3,poly_sigma=5,flags=cv2.OPTFLOW_FARNEBACK_GAUSSIAN)
    magnitude, angle = cv2.cartToPolar(flow[..., 0], flow[..., 1])
    return cv2.normalize(magnitude, None, 0, 255, cv2.NORM_MINMAX)[0]

def calculate_opticalFlow_scene(scene):
    first = calculate_opticalFlow(scene[0], scene[1])
    second = calculate_opticalFlow(scene[1], scene[2])
    return np.mean([first, second])
    
def isjpg(filepath):
    return re.search(".jpg$", filepath)

def calculate_features(img_path):
    img = cv2.imread(img_path)
    b = mean_brightness(img)
    s = mean_saturation(img)
    result = calculate_pad(b, s)
    result.append(calculate_blur(img))
    return result

In [3]:
base = 'data\\outside\\training'
dirs = os.listdir(base)
labels = []
train_ps = []
train_as = []
train_ds = []
train_blurs = []
for emotion in dirs:
    emotion_path = os.path.join(base, emotion)
    for img in tqdm_notebook(os.listdir(emotion_path)):
        img_path = os.path.join(emotion_path, img)
        res = calculate_features(img_path)
        labels.append(emotion)
        train_ps.append(res[0])
        train_as.append(res[1])
        train_ds.append(res[2])
        train_blurs.append(res[3])

In [4]:
df = pd.DataFrame()
df['label'] = labels
df['valence'] = train_ps
df['arousal'] = train_as
df['dominance'] = train_ds
df['blur'] = train_blurs

In [5]:
X = df.drop(columns=['label'])
X = (X - X.min())/(X.max() - X.min())
Y = df.label
clf = LogisticRegression(random_state=0).fit(X, Y)

C:\Users\xinrui zhan\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\xinrui zhan\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


In [6]:
our_dataset = pd.read_csv('clean_df.csv')
our_X = our_dataset[['scene_avg_p', 'scene_avg_a', 'scene_avg_d', 'scene_avg_blur']]
our_X = (our_X - our_X.min())/(our_X.max() - our_X.min())
prd = clf.predict(our_X)

In [7]:
our_dataset['predict'] = prd

In [8]:
our_dataset.to_csv('logistic_regression_trained_on_outside.csv', index=False)

In [9]:
temp = pd.read_csv('baseline_clean_df.csv')
temp

,Unnamed: 0,scene_name,scene_avg_p,scene_avg_a,scene_avg_d,scene_avg_blur,scene_avg_optical_flow,scene_movie,Emotion
0,0,big_hero_60,64.405538,72.981351,82.268261,204.102944,4.284929e-08,big_hero_6,"['Surprise', 'Joy', 'Anger']"
1,1,big_hero_61,76.658546,45.019868,93.268124,2058.214860,4.400276e-02,big_hero_6,"['Joy', 'Surprise', 'Anger']"
2,2,big_hero_610,93.844222,48.897762,113.486587,319.563208,7.499705e-03,big_hero_6,"['Joy', 'Surprise', 'Anger']"
3,3,big_hero_611,104.689522,68.900088,128.197153,344.318350,3.311605e-06,big_hero_6,"['Joy', 'Surprise', 'Anger']"
4,4,big_hero_612,52.825514,48.692155,66.235224,302.668920,1.479887e-03,big_hero_6,"['Surprise', 'Joy', 'Anger']"
5,5,big_hero_613,54.635902,56.085796,69.141541,373.148184,1.438304e-03,big_hero_6,"['Surprise', 'Joy', 'Anger']"
6,6,big_hero_614,60.207244,39.892563,73.756332,187.768550,1.667552e-03,big_hero_6,"['Surprise', 'Joy', 'Anger']"
7,7,big_hero_615,51.286053,48.994812,64.496347,81.434549,3.594452e-05,big_hero_6,"['Surprise', 'Joy', 'Anger']"
8,8,big_hero_616,45.333909,38.411284,56.466676,110.447071,NaN,big_hero_6,"['Surprise', 'Joy', 'Anger']"
9,9,big_hero_617,66.545265,37.953094,80.838276,609.955143,9.923468e-04,big_hero_6,"['Joy', 'Surprise', 'Anger']"


In [40]:
df = pd.read_csv('clean_df.csv')

In [43]:
df.paths_1[0]

'data\\scenes\\big_hero_6\\big_hero_6-Scene-0-0.jpg'

In [ ]:
cv2.imread()